In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Redback_A/toxic.csv")
df = df.drop("id", axis=1)
df.columns = ['comment', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df.head()

,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
## The data is very imbalanced
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
toxic_comments = df[df[classes].sum(axis=1)>0]
clean_comments = df[df[classes].sum(axis=1)==0]

print("Number of toxic comments", len(toxic_comments))
print("Number of clean comments", len(clean_comments))

Number of toxic comments 16225
Number of clean comments 143346


In [6]:
## Sample clean comments and make dataset balanced
uniform_dataset = pd.concat([toxic_comments, clean_comments.sample(len(toxic_comments))])

In [7]:
## The data is now balanced
toxic_comments = uniform_dataset[uniform_dataset[classes].sum(axis=1)>0]
clean_comments = uniform_dataset[uniform_dataset[classes].sum(axis=1)==0]

print("Number of toxic comments", len(toxic_comments))
print("Number of clean comments", len(clean_comments))

Number of toxic comments 16225
Number of clean comments 16225


In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.1 MB/s eta 0:00:00


In [11]:
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

X = uniform_dataset["comment"]
y = uniform_dataset.iloc[:, 1:]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Tokenize the text data using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
X_test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

In [13]:
# Convert the encoded features and labels into PyTorch tensors
X_train_input_ids = torch.tensor(X_train_encodings['input_ids'])
X_train_attention_mask = torch.tensor(X_train_encodings['attention_mask'])
y_train = torch.tensor(y_train.values, dtype=torch.float32)

X_test_input_ids = torch.tensor(X_test_encodings['input_ids'])
X_test_attention_mask = torch.tensor(X_test_encodings['attention_mask'])
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [14]:
# Create a PyTorch DataLoader object for training and testing data
train_dataset = TensorDataset(X_train_input_ids, X_train_attention_mask, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test_input_ids, X_test_attention_mask, y_test)
test_loader = DataLoader(test_dataset, batch_size=16)

In [15]:
# Create a DistilBERT-based multilabel classification model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
# Count the number of batches in train_loader
import math

batch_size = 16  # set the batch size

num_batches = math.ceil(len(train_dataset) / batch_size)
print(f"Number of batches in train_loader: {num_batches}")

Number of batches in train_loader: 1420


In [ ]:
import random
import torch.nn as nn

# Train the model using a suitable optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 1
batch_size = 32
train_subset_size = 500  # number of batches to use for training
random_seed = 42  # set a random seed for reproducibility
run_count = 0 # to count the number of runs in the loop

model.train()

criterion = nn.BCEWithLogitsLoss()

# set the random seed
random.seed(random_seed)

# convert train_loader to a list of batches
train_batches = list(train_loader)


for epoch in range(num_epochs):
    # randomly select a subset of batches for training
    train_subset = random.sample(train_batches, train_subset_size)
    
    for batch in train_subset:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)

        loss = criterion(outputs.logits, labels.float())

        loss.backward()

        optimizer.step()
        # compute and print the current loss
        run_count += 1
        current_loss = loss.item()
        print(f"Current loss: {run_count}: {current_loss:.4f}")

In [24]:
# Set the model to evaluation mode
model.eval()

# Create empty lists for storing true labels and predicted labels
true_labels = []
pred_labels = []

# Iterate through the test data and make predictions
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)

        # Convert the logits to probabilities and round off to 0 or 1
        probs = torch.sigmoid(outputs.logits)
        preds = (probs >= 0.5).float()

        true_labels.extend(labels.cpu().detach().numpy().tolist())
        pred_labels.extend(preds.cpu().detach().numpy().tolist())

In [25]:
# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='micro')
recall = recall_score(true_labels, pred_labels, average='micro')
f1 = f1_score(true_labels, pred_labels, average='micro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

Accuracy: 0.7073446327683616
Precision: 0.8249976541240499
Recall: 0.835027068097635
F1 score: 0.8299820636269233
